# Rectification After Camera orientation correction using vanishing point estimation
### Author: Muhammad Rafi Faisal

In [1]:
import os                      
import cv2                     
import math                    
import numpy as np             
import random                  
import matplotlib.pyplot as plt

# Function for feature detection (Super Point), matching (Super Glue), and RANSAC-based pose estimation
source: https://github.com/magicleap/SuperGluePretrainedNetwork

In [2]:
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
from models.superpoint import SuperPoint
from models.superglue import SuperGlue


def visualize_matches(img1, img2, points1, points2, inlier_mask):
    if len(img1.shape) == 2:
        img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    if len(img2.shape) == 2:
        img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)

    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    height = max(h1, h2)
    width = w1 + w2
    combined = np.zeros((height, width, 3), dtype=np.uint8)
    combined[:h1, :w1] = img1
    combined[:h2, w1:w1+w2] = img2

    for i in range(len(points1)):
        pt1 = (int(points1[i][0]), int(points1[i][1]))
        pt2 = (int(points2[i][0] + w1), int(points2[i][1]))
        color = (0, 255, 0) if inlier_mask[i] else (0, 0, 255)
        cv2.line(combined, pt1, pt2, color, 1)
        cv2.circle(combined, pt1, 2, color, -1)
        cv2.circle(combined, pt2, 2, color, -1)
        
    plt.figure(figsize=(12, 6))
    plt.imshow(cv2.cvtColor(combined, cv2.COLOR_BGR2RGB))
    plt.title("Matches: Green = Inliers, Red = Outliers")
    plt.axis('off')
    plt.show()


def feature_detection_and_matching(images0, images1):
    all_points_1 = []
    all_points_2 = []
    
    for i in range(len(images1)):
        img1, img2 = images0[i], images1[i]

        # Detect features using SuperPoint
        kp1, desc1, _ = detect_features_with_superpoint(img1)
        kp2, desc2, _ = detect_features_with_superpoint(img2)

        # Match features using SuperGlue
        points_1, points_2 = match_features_with_superglue(kp1, desc1, img1, kp2, desc2, img2)

        if len(points_1) < 8 or len(points_2) < 8:
            print(f"Not enough matches found for Image Pair: {i}")
            continue
            
        all_points_1.append(points_1)
        all_points_2.append(points_2)
    
    return all_points_1, all_points_2


# Running with SUPER POINT and GLUE

In [3]:
import itertools
import torch
import numpy as np
import cv2
from models.superpoint import SuperPoint
from models.superglue import SuperGlue

# Initialize device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load SuperPoint & SuperGlue models
config = {
    'superpoint': {'nms_radius': 4, 'keypoint_threshold': 0.005, 'max_keypoints': -1}, # threshold 0.001/0.005
    'superglue': {'weights': 'indoor', 'sinkhorn_iterations': 20} # changing now weights from "indoor" to "outdoor"
}
superpoint = SuperPoint(config['superpoint']).to(device).eval()
superglue = SuperGlue(config['superglue']).to(device).eval()

feature_detectors = [
    superpoint,
]

feature_matchers = [
    superglue, 
]

# Camera matrix (adjust based on dataset)
# camera_matrix = np.array([   # Budapest dataset
#     [1250, 0, 960],
#     [0, 1250, 600],
#     [0, 0, 1]
# ])

# camera_matrix = np.array([[718.856,    0,    607.1928], #kitti
#  [  0,     718.856,  185.2157],
#  [  0,       0,       1    ]])

results, results_a, results_b = [], [], []
points3d_list, points2d_list, pose_list = [], [], []

def detect_features_with_superpoint(image):
    #Extracts keypoints & descriptors using SuperPoint
    if len(image.shape) == 3 and image.shape[2] == 3:
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        image_gray = image
        
    # image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_tensor = torch.from_numpy(image_gray).float()[None, None] / 255.0  # Normalize
    image_tensor = image_tensor.to(device)

    with torch.no_grad():
        outputs = superpoint({'image': image_tensor})

    keypoints = outputs['keypoints'][0].cpu().numpy()
    descriptors = outputs['descriptors'][0].cpu().numpy()
    scores = outputs['scores'][0].cpu().numpy()
    print(f"Detected {len(keypoints)} keypoints")
    return keypoints, descriptors, scores

def match_features_with_superglue(kp1, desc1, img1, kp2, desc2, img2):
    # Convert images to grayscale and normalize; shape: [1, 1, H, W]
    if len(img1.shape) == 3 and img1.shape[2] == 3:
        img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        img2_gray = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    else:
        img1_gray = img1
        img2_gray = img2
    # img1_gray = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    # img2_gray = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    img1_tensor = torch.from_numpy(img1_gray).float()[None, None] / 255.0
    img2_tensor = torch.from_numpy(img2_gray).float()[None, None] / 255.0
    img1_tensor = img1_tensor.to(device)
    img2_tensor = img2_tensor.to(device)
    
    # Create data dictionary with actual images
    data = {
        'image0': img1_tensor,
        'image1': img2_tensor,
        'keypoints0': torch.tensor(kp1, dtype=torch.float32)[None].to(device),
        'keypoints1': torch.tensor(kp2, dtype=torch.float32)[None].to(device),
        'descriptors0': torch.tensor(desc1, dtype=torch.float32)[None].to(device),
        'descriptors1': torch.tensor(desc2, dtype=torch.float32)[None].to(device),
        'scores0': torch.ones((1, kp1.shape[0]), device=device),
        'scores1': torch.ones((1, kp2.shape[0]), device=device)
    }
    
    with torch.no_grad():
        outputs = superglue(data)
    
    matches = outputs['matches0'][0].cpu().numpy()
    valid_matches = matches > -1
    num_matches = valid_matches.sum()
    print(f"Found {num_matches} valid matches")
    
    if num_matches == 0:
        return np.empty((0, 2)), np.empty((0, 2))
    
    matched_kp1 = kp1[valid_matches]
    matched_kp2 = kp2[matches[valid_matches]]
    
    return matched_kp1, matched_kp2



# for feature_detector, matcher in itertools.product(feature_detectors, feature_matchers):
#     try:
#         result, result_a, result_b= feature_detection_and_matching(images0, images1)

#         results.append((feature_detector, matcher, result))
#         results_a.append((feature_detector, matcher, result_a))
#         results_b.append((feature_detector, matcher, result_b))

#     except Exception as e:
#         print(f"Error with combination: {feature_detector} and {matcher}: {e}")


/home/rafi/1_vanishing_point_est/SuperGluePretrainedNetwork/models/superpoint.py:137: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(str(path)

Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)


/home/rafi/1_vanishing_point_est/SuperGluePretrainedNetwork/models/superglue.py:226: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(str(path))

In [4]:
import numpy as np
# import cv2 as cv
from matplotlib import pyplot as plt
import sys
import glob

#Draw epipolar lines:
#
#img1: first image
#img1: second image
#
#lines: corresponding epilines
#pts1: point in first image
#pts2: point in second image
#
#colors: array of random colors for visualization
#
#Return: img1, img2 original images with drawn epilines

def drawlines(img1,img2,lines,pts1,pts2,colors,F):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''

    print(img1.shape)

#r: image height
#c: image width
    r,c  = img1.shape
    img1 = cv2.cvtColor(img1,cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2,cv2.COLOR_GRAY2BGR)
    cnt=0;
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color=colors[cnt,:].tolist()

#Get points in homogeneous form
        pt1Hom=np.ones(3);
        pt1Hom[0]=pt1[0]
        pt1Hom[1]=pt1[1]

        pt2Hom=np.ones(3);
        pt2Hom[0]=pt2[0]
        pt2Hom[1]=pt2[1]

#Compute epipolar lines in the first image
#        r1=F@pt1Hom
        r1=F.transpose()@pt2Hom

        x0,y0 = map(int, [0, -r1[2]/r1[1] ])
        x1,y1 = map(int, [c, -(r1[2]+r1[0]*c)/r1[1] ])

#Draw epipolar line
        img1 = cv2.line(img1, (x0,y0), (x1,y1), color,1)
#Draw feature point location
        img1 = cv2.circle(img1,tuple(pt1),5,color,-1)
        img2 = cv2.circle(img2,tuple(pt2),5,color,-1)
        cnt=cnt+1
    return img1,img2

## For single img

In [21]:
#Load image
# fileName1=sys.argv[1]
# fileName2=sys.argv[2]

img1 = cv2.imread('../../dataset_budapest/elte_car_stereo/dev0_70_520/Dev0_Image_w1920_h1200_fn457.jpg', cv2.IMREAD_GRAYSCALE)  #queryimage # left image #, cv.IMREAD_GRAYSCALE
img2 = cv2.imread('../../dataset_budapest/elte_car_stereo/dev1_70_520/Dev1_Image_w1920_h1200_fn457.jpg', cv2.IMREAD_GRAYSCALE) #trainimage # right image
# LEFT_PATH  = "../../dataset_budapest/elte_car_stereo/dev0_70_520/Dev0_Image_w1920_h1200_fn100.jpg"
# RIGHT_PATH = "../../dataset_budapest/elte_car_stereo/dev1_70_520/Dev1_Image_w1920_h1200_fn100.jpg"
# img1 = cv2.imread('../output/dataset_2/img_l_corr/Dev0_Image_w1920_h1200_fn130.jpg', cv2.IMREAD_GRAYSCALE)  #queryimage # left image #, cv.IMREAD_GRAYSCALE
# img2 = cv2.imread('../output/dataset_2/img_r_corr/Dev1_Image_w1920_h1200_fn130.jpg', cv2.IMREAD_GRAYSCALE) #trainimage # right image

#img1 = cv.imread('Dev0_Image_w1920_h1200_fn400.jpg', cv.IMREAD_GRAYSCALE)  #queryimage # left image
#img2 = cv.imread('Dev1_Image_w1920_h1200_fn400.jpg', cv.IMREAD_GRAYSCALE) #trainimage # right image

pts1 = []
pts2 = []
#Apply SIFT feature detection
#Also retreive the desriptiors
pts1, pts2 = feature_detection_and_matching([img1], [img2])

pts1= np.int32(pts1[0])
pts2 = np.int32(pts2[0])


#Estimate fundamental matrix using RANSAC.
#(Point normalization included)
F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_RANSAC)
# We select only inlier points
pts1 = pts1[mask.ravel()==1]
pts2 = pts2[mask.ravel()==1]


#Number of inliers:

NUM_INLIERS=pts1.shape[0]

#Generate random colorrs for visualization
colors=np.zeros((NUM_INLIERS,3))
for idx in range(0,NUM_INLIERS):
    colors[idx,0]=np.random.randint(0,255)
    colors[idx,1]=np.random.randint(0,255)
    colors[idx,2]=np.random.randint(0,255)


# Find epilines corresponding to points in right image (second image) and
# drawing its lines on left image
lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2,F)
lines1 = lines1.reshape(-1,3)
img5,img6 = drawlines(img1,img2,lines1,pts1,pts2,colors,F)


# Find epilines corresponding to points in left image (first image) and
# drawing its lines on right image
lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
lines2 = lines2.reshape(-1,3)
img3,img4 = drawlines(img2,img1,lines2,pts2,pts1,colors,F.transpose())

# cv.imshow("Matches", img5)
# cv.waitKey(30000)

cv2.imwrite("rect_results/EpipLines1.png",img3)
cv2.imwrite("rect_results/Features1.png",img4)
cv2.imwrite("rect_results/EpipLines2.png",img5)
cv2.imwrite("rect_results/Features2.png",img6)


np.savetxt("rect_results/F_RANSAC.mat",F)
np.savetxt("rect_results/pts1.mat",pts1)
np.savetxt("rect_results/pts2.mat",pts2)


Detected 2962 keypoints
Detected 1731 keypoints
Found 172 valid matches
(1200, 1920)
(1200, 1920)


## For all images

In [8]:
img_l_files = sorted(glob.glob('../../dataset_budapest/elte_car_stereo/dev0_70_520/*.jpg'))
img_r_files = sorted(glob.glob('../../dataset_budapest/elte_car_stereo/dev1_70_520/*.jpg'))

for i in range(len(img_l_files)):
    # Read images in grayscale
    img1 = cv2.imread(img_l_files[i], cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(img_r_files[i], cv2.IMREAD_GRAYSCALE)
    
    if img1 is None or img2 is None:
        print(f"Error reading image pair {i}")
        continue

    # Get feature points from both images
    pts1_list, pts2_list = feature_detection_and_matching([img1], [img2])
    pts1 = pts1_list[0]
    pts2 = pts2_list[0]
    
    # Ensure there are enough points for estimation
    if len(pts1) < 8 or len(pts2) < 8:
        print(f"Not enough feature matches for pair {i}")
        continue

    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)

    # Estimate the fundamental matrix using RANSAC
    F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC)
    pts1 = pts1[mask.ravel() == 1]
    pts2 = pts2[mask.ravel() == 1]

    # Number of inliers:
    NUM_INLIERS = pts1.shape[0]
    if NUM_INLIERS == 0:
        print(f"No inliers found for pair {i}")
        continue

    # Generate random colors for each inlier point
    colors = np.random.randint(0, 255, (NUM_INLIERS, 3))

    # Compute epipolar lines for each image
    lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2, F)
    lines1 = lines1.reshape(-1,3)
    img_epilines1, img_features1 = drawlines(img1, img2, lines1, pts1, pts2, colors, F)

    lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1, F)
    lines2 = lines2.reshape(-1,3)
    img_epilines2, img_features2 = drawlines(img2, img1, lines2, pts2, pts1, colors, F.transpose())

    # Save results as images
    output_dir = "rect_results"
    # cv2.imwrite(f"{output_dir}/EpipLines1_pair_{i}.png", img_epilines1)
    # cv2.imwrite(f"{output_dir}/Features1_pair_{i}.png", img_features1)
    # cv2.imwrite(f"{output_dir}/EpipLines2_pair_{i}.png", img_epilines2)
    # cv2.imwrite(f"{output_dir}/Features2_pair_{i}.png", img_features2)

    # Save fundamental matrix and feature points as text files
    np.savetxt(f"{output_dir}/F_RANSAC_pair_{i}.txt", F)
    np.savetxt(f"{output_dir}/pts1_pair_{i}.txt", pts1)
    np.savetxt(f"{output_dir}/pts2_pair_{i}.txt", pts2)

    print(f"Processed pair {i}: saved images and txt files.")

Detected 6794 keypoints
Detected 3976 keypoints
Found 2402 valid matches
(1200, 1920)
(1200, 1920)
Processed pair 0: saved images and txt files.
Detected 6956 keypoints
Detected 3951 keypoints
Found 2477 valid matches
(1200, 1920)
(1200, 1920)
Processed pair 1: saved images and txt files.
Detected 6942 keypoints
Detected 4029 keypoints
Found 2501 valid matches
(1200, 1920)
(1200, 1920)
Processed pair 2: saved images and txt files.
Detected 7129 keypoints
Detected 4009 keypoints
Found 2482 valid matches
(1200, 1920)
(1200, 1920)
Processed pair 3: saved images and txt files.
Detected 7152 keypoints
Detected 4002 keypoints
Found 2448 valid matches
(1200, 1920)
(1200, 1920)
Processed pair 4: saved images and txt files.
Detected 7121 keypoints
Detected 3994 keypoints
Found 2538 valid matches
(1200, 1920)
(1200, 1920)
Processed pair 5: saved images and txt files.
Detected 7096 keypoints
Detected 4034 keypoints
Found 2411 valid matches
(1200, 1920)
(1200, 1920)
Processed pair 6: saved images 

## Now Rectification

In [22]:
import numpy as np
# import cv2 as cv
import math
import sys

#TransformCorners: transforms corner points by a Homography
#
#H: Homography
#(WIDTH,HEIGHT): image sizes
#
#Return: 3x4 matrix with the transformed corners. Points are given in homogeneous coordinates.
def TransformCorners(H,WIDTH,HEIGHT):
    pts=np.ones((3,4))
#Corners:
    pts[0,0]=0;
    pts[1,0]=0;

    pts[0,1]=WIDTH;
    pts[1,1]=0;

    pts[0,2]=0;
    pts[1,2]=HEIGHT;

    pts[0,3]=WIDTH;
    pts[1,3]=HEIGHT;
#Transformation
    pts=H1@pts
    return pts

#H1: homography for first image
#(WIDTH1,HEIGHT1): size of first image
#
#H2: homography for first image
#(WIDTH2,HEIGHT2): size of first image
#(EXPECTED_WIDTH,EXPECTED_HEIGHT): expected sizef of the resulting rectified images
#
#Return: Correction homography


def FinalScaleOffset(H1,WIDTH1,HEIGHT1,H2,WIDTH2,HEIGHT2,EXPECTED_WIDTH,EXPECTED_HEIGHT):

    ptsIm=np.ones((8,2))

    pts1Hom=TransformCorners(H1,WIDTH1,HEIGHT1)
    pts2Hom=TransformCorners(H2,WIDTH2,HEIGHT2)

    print("pts1Hom:",pts1Hom)
    print("pts2Hom:",pts2Hom)

    # print(pts1Hom)

    for idx in range(0,4):
        pt1=pts1Hom[0:2,idx]/pts1Hom[2,idx]
        pt2=pts2Hom[0:2,idx]/pts2Hom[2,idx]

        ptsIm[idx,:]=pt1
        ptsIm[idx+4,:]=pt1

    minPts=ptsIm.min(0)
    maxPts=ptsIm.max(0)

    print("Pts:",ptsIm)

    print("minPts",minPts)
    print("maxPts",maxPts)

    alphaW=EXPECTED_WIDTH/(maxPts[0]-minPts[0])
    betaW=-1.0*alphaW*minPts[0]

    alphaH=EXPECTED_HEIGHT/(maxPts[1]-minPts[1])
    betaH=-1.0*alphaH*minPts[1]

    Corr=np.eye((3))
    Corr[0,0]=alphaW
    Corr[1,1]=alphaH
    Corr[0,2]=betaW
    Corr[1,2]=betaH

    return Corr


#Estimate the verticalScaleand Offset
#Fmod
#pts1Hom,pts2Hom: Points after transformation in homogeneous coordinates
#Return: Desired transformation
#
#The scale and offset is given as y1=alpha*y2+beta2
#

def EstimateVerticalOffsetAndScale(Fmod,pts1Hom,pts2Hom):

#Number of points
    NUMP=pts1Hom.shape[1]

#Each point gives an equation for alpha and beta
    Cfs=np.zeros((NUMP,2))
    right=np.zeros((NUMP,1))

    for idx in range(0,NUMP):
#Get original homogeneous coordinates:
        p1Hom=pts1Hom[:,idx]
        p2Hom=pts2Hom[:,idx]
#y2:
        Cfs[idx,0]=p2Hom[1]/p2Hom[2]  # coordinate x
        Cfs[idx,1]=1.0

        right[idx,0]=p1Hom[1]/p1Hom[2]

#Over-determined inhomogeneous linear system of equations
    params=np.linalg.inv(Cfs.transpose()@Cfs)@Cfs.transpose()@right
    alpha=params[0]
    beta=params[1]


#First transformation is the identity, second one is the copmputed scale/offset
    H1=np.eye((3))
    H2=np.eye((3))

    H2[1,1]=alpha
    H2[1,2]=beta

    return H1,H2


#This method is the essence of rectification
#It moves the epipole to the infinity
#
#Input:
#
#epipole: computed epipole. It is input even if it can be computed as the kernel of F.
#F: fundamental matrix
#
#Return:
#Resulting homography

def EstimateRectifyingHomography(epipole,F):

#In the first step, the vertical coordinate of the epipole should be moved to y=0
    tform1=np.eye((3))
    tform1[1,2]=-epipole[1]

    #Tform 2 is the projective warp
    tform2=np.eye((3))
    tform2[2,0]=-1.0/epipole[0];

    H=tform2@tform1

    return H


## For single image

In [23]:
# fileName1=sys.argv[1]
# fileName2=sys.argv[2]

# img1 = cv.imread(fileName1)  #queryimage # left image
# img2 = cv.imread(fileName1) #trainimage # right image

# img1 = cv2.imread('a.png')  #queryimage # left image
# img2 = cv2.imread('b.png') #trainimage # right image

img1 = cv2.imread('../../dataset_budapest/elte_car_stereo/dev0_70_520/Dev0_Image_w1920_h1200_fn457.jpg', cv2.IMREAD_GRAYSCALE)  #queryimage # left image #, cv.IMREAD_GRAYSCALE
img2 = cv2.imread('../../dataset_budapest/elte_car_stereo/dev1_70_520/Dev1_Image_w1920_h1200_fn457.jpg', cv2.IMREAD_GRAYSCALE)

#Copy original images. It will be saved later

origImg1=img1.copy()
origImg2=img2.copy()



#Image sizes:
WIDTH1=img1.shape[1]
HEIGHT1=img1.shape[0]

WIDTH2=img2.shape[1]
HEIGHT2=img2.shape[0]


#White:
color=np.ones(3)


#Load fundamental matrix and matched (inlier) feature point:
F=np.loadtxt("rect_results/F_RANSAC.mat")
pts1=np.loadtxt("rect_results/pts1.mat")
pts2=np.loadtxt("rect_results/pts2.mat")


#Compute epipoles

u,s,vt= np.linalg.svd(F)

#Left and right kernels give epipoles
epipole1=vt[2,:]
epipole2=u[:,2]

#Homogeneous division required!
epipole1=epipole1/epipole1[2]
epipole2=epipole2/epipole2[2]


#For debug:
print("Epipole1:",epipole1[0:2])
print("Epipole2:",epipole2[0:2])



##Parameter to visualize epipoles. Distance between drawn lines
SKIP=10

#Number of drawn epilines
NUM=int(math.floor(HEIGHT1/SKIP))



#Take and draw epipolar lines in the second image
for idx in range(0,NUM):
    ptHom=np.ones(3)
#Center of width
    ptHom[0]=int(WIDTH1/2)
#Uniform sampling:
    ptHom[1]=idx*SKIP

#Epipolar line in the second image
    line2=F@ptHom

#Get endpoints of line in the screen.
    x0,y0 = map(int, [0, -line2[2]/line2[1] ])
    x1,y1 = map(int, [WIDTH2, -(line2[2]+line2[0]*WIDTH2)/line2[1] ])
#Draw it
    img2 = cv2.line(img2, (x0,y0), (x1,y1), color,1)



#Take and draw epipolar lines in the first image image

NUM=int(math.floor(HEIGHT2/SKIP))

for idx in range(0,NUM):
    ptHom=np.ones(3)
    ptHom[1]=idx*SKIP
    ptHom[0]=int(WIDTH2/2)

#F.transpose() gives the inverz of the fundamental matrix!
    line1=F.transpose()@ptHom

    x0,y0 = map(int, [0, -line1[2]/line1[1] ])
    x1,y1 = map(int, [WIDTH1, -(line1[2]+line1[0]*WIDTH1)/line1[1] ])
    img1 = cv2.line(img1, (x0,y0), (x1,y1), color,1)




#Write images for debgging purposes

cv2.imwrite("rect_results/res1.png",img1)
cv2.imwrite("rect_results/res2.png",img2)


#Move epipoles to the infinity:
H1=EstimateRectifyingHomography(epipole1,F)
H2=EstimateRectifyingHomography(epipole2,F.transpose())


tmp1 = cv2.warpPerspective(origImg1, H1,(WIDTH1,HEIGHT1))
tmp2 = cv2.warpPerspective(origImg2, H2,(WIDTH2,HEIGHT2))

cv2.imwrite("rect_results/tmp1.png",tmp1)
cv2.imwrite("rect_results/tmp2.png",tmp2)


#Modified fundamental matrix
Fmod=np.linalg.inv(H2.transpose())@F@np.linalg.inv(H1)



#Transform the point coordinates by the obtrained homography

#Number of points
NUMP=pts1.shape[0]

#Point should be considered in its homogeneous form
pts1Hom=np.ones((NUMP,3))
pts2Hom=np.ones((NUMP,3))




for idx in range(0,NUMP):
    pts1Hom[idx,0]=pts1[idx,0]
    pts1Hom[idx,1]=pts1[idx,1]

    pts2Hom[idx,0]=pts2[idx,0]
    pts2Hom[idx,1]=pts2[idx,1]



pts1Hom=H1@(pts1Hom.transpose())
pts2Hom=H2@(pts2Hom.transpose())


#Make horizontal epipolar lines parallel by estimating a scale and an offset
H1second,H2second=EstimateVerticalOffsetAndScale(Fmod,pts1Hom,pts2Hom)

H1=H1second@H1
H2=H2second@H2



#Finally, move the image to the center and resize the images

EXPECTED_WIDTH=960
EXPECTED_HEIGHT=600 #600


Corr= FinalScaleOffset(H1,WIDTH1,HEIGHT1,H2,WIDTH2,HEIGHT2,EXPECTED_WIDTH,EXPECTED_HEIGHT)


H1=Corr@H1
H2=Corr@H2


## Calculations for doffs
# camera_matrix = np.array([   # Budapest dataset
#     [1250, 0, 960],
#     [0, 1250, 600],
#     [0, 0, 1]
# ])

camera_matrix = np.array([   # Budapest dataset
    [625, 0, 480],
    [0, 625, 300],
    [0, 0, 1]
])
p1 = np.array([camera_matrix[0, 2], camera_matrix[1, 2], 1])  # [960, 600, 1]
p2 = np.array([camera_matrix[0, 2], camera_matrix[1, 2], 1])

p1_rect = H1 @ p1
p2_rect = H2 @ p2

# Normalize the homogeneous coordinates
p1_rect = p1_rect / p1_rect[2]
p2_rect = p2_rect / p2_rect[2]

# Calculate disparity offset (doffs)
doffs = p2_rect[0] - p1_rect[0]
print("Disparity offset (doffs):", doffs)


#Finally, draw image for debugging purposes:


transformed1 = cv2.warpPerspective(img1, H1,(EXPECTED_WIDTH,EXPECTED_HEIGHT))
transformed2 = cv2.warpPerspective(img2, H2,(EXPECTED_WIDTH,EXPECTED_HEIGHT))

cv2.imwrite("rect_results/transformed1.png",transformed1)
cv2.imwrite("rect_results/transformed2.png",transformed2)

#Rectify original images.

result1 = cv2.warpPerspective(origImg1, H1,(EXPECTED_WIDTH,EXPECTED_HEIGHT))
result2 = cv2.warpPerspective(origImg2, H2,(EXPECTED_WIDTH,EXPECTED_HEIGHT))

cv2.imwrite("rect_results/final1.png",result1)
cv2.imwrite("rect_results/final2.png",result2)

#Concatenate resulting images:
final=cv2.hconcat([result1,result2])
cv2.imwrite("rect_results/finalRectified.png",final)

final=cv2.hconcat([origImg1,origImg2])
cv2.imwrite("rect_results/origRectified.png",final)

Epipole1: [6971.1238279   546.33332895]
Epipole2: [8069.84973282  322.84901382]
pts1Hom: [[ 0.00000000e+00  1.92000000e+03  0.00000000e+00  1.92000000e+03]
 [-5.46333329e+02 -5.46333329e+02  6.53666671e+02  6.53666671e+02]
 [ 1.00000000e+00  7.24578124e-01  1.00000000e+00  7.24578124e-01]]
pts2Hom: [[ 0.00000000e+00  1.92000000e+03  0.00000000e+00  1.92000000e+03]
 [-5.46333329e+02 -5.46333329e+02  6.53666671e+02  6.53666671e+02]
 [ 1.00000000e+00  7.24578124e-01  1.00000000e+00  7.24578124e-01]]
Pts: [[   0.         -546.33332895]
 [2649.81778424 -754.00196415]
 [   0.          653.66667105]
 [2649.81778424  902.134151  ]
 [   0.         -546.33332895]
 [2649.81778424 -754.00196415]
 [   0.          653.66667105]
 [2649.81778424  902.134151  ]]
minPts [   0.         -754.00196415]
maxPts [2649.81778424  902.134151  ]
Disparity offset (doffs): -1.8615469315089683


True

## For all images

In [11]:
import cv2
import numpy as np
import math
import glob
import os

# # Define your camera matrix (example for the Budapest dataset)
camera_matrix_01 = np.array([   # elte_car_stereo, cam0
    [1.1049e+03,-263.4241,1.1561e+03],
    [0,1.1799e+03,235.7677],
    [0, 0, 1]
])

camera_matrix_02 = np.array([   # elte_car_stereo, cam1 
    [4.9384e+03,-363.6950,1.1743e+03],
    [0,4.5852e+03,809.9989],
    [0, 0, 1]
])
# camera_matrix = np.array([   # Budapest dataset
#     [625, 0, 480],
#     [0, 625, 300],
#     [0, 0, 1]
# ])
# camera_matrix_01 =	np.array([[1.2799e+03,0,1.0148e+03] , [0,1.2841e+03,573.2751] , [0,0,1]])

# camera_matrix_02 = 	np.array([[1.2706e+03,0,961.2221] , [0,1.2739e+03,611.5167] , [0,0,1]])


camera_matrix_01_r = camera_matrix_01 / 2
camera_matrix_02_r = camera_matrix_02 / 2

# Directories for left/right images (update extensions if needed)
left_img_files = sorted(glob.glob('../../dataset_budapest/elte_car_stereo/dev0_70_520/*.jpg'))
right_img_files = sorted(glob.glob('../../dataset_budapest/elte_car_stereo/dev1_70_520/*.jpg'))

# Directory where the F and pts files are saved
results_dir = "rect_results"
results_dir_l = "rect_results/dev0"
results_dir_r = "rect_results/dev1"
doffs_array = []
# Get the number of pairs (assuming the ordering of left/right is matching)
num_pairs = min(len(left_img_files), len(right_img_files))

for i in range(num_pairs):
    print(f"Processing pair {i}")
    # Read left and right images
    img1 = cv2.imread(left_img_files[i])
    img2 = cv2.imread(right_img_files[i])
    if img1 is None or img2 is None:
        print(f"Error reading images for pair {i}")
        continue

    # Save original images for later warping
    origImg1 = img1.copy()
    origImg2 = img2.copy()

    # Image sizes
    WIDTH1, HEIGHT1 = img1.shape[1], img1.shape[0]
    WIDTH2, HEIGHT2 = img2.shape[1], img2.shape[0]

    # White color for drawing (you can adjust as needed)
    color = np.ones(3)  # This creates a [1,1,1] vector (if needed, multiply by 255)

    # Read the fundamental matrix and inlier feature points for this pair
    F_path = os.path.join(results_dir, f"F_RANSAC_pair_{i}.txt")
    pts1_path = os.path.join(results_dir, f"pts1_pair_{i}.txt")
    pts2_path = os.path.join(results_dir, f"pts2_pair_{i}.txt")
    try:
        F = np.loadtxt(F_path)
        pts1 = np.loadtxt(pts1_path)
        pts2 = np.loadtxt(pts2_path)
    except Exception as e:
        print(f"Error loading F/pts for pair {i}: {e}")
        continue

    # --- Compute epipoles ---
    u, s, vt = np.linalg.svd(F)
    epipole1 = vt[2, :]
    epipole2 = u[:, 2]
    epipole1 = epipole1 / epipole1[2]
    epipole2 = epipole2 / epipole2[2]
    print(f"Pair {i} - Epipole1: {epipole1[0:2]}, Epipole2: {epipole2[0:2]}")

    # --- Draw epipolar lines on the images for debugging ---
    SKIP = 10  # vertical spacing between sampled lines

    # Draw epipolar lines in the second image based on points in the first image
    NUM = int(math.floor(HEIGHT1 / SKIP))
    for idx in range(NUM):
        ptHom = np.ones(3)
        ptHom[0] = int(WIDTH1 / 2)
        ptHom[1] = idx * SKIP
        line2 = F @ ptHom
        if line2[1] != 0:
            x0, y0 = 0, int(-line2[2] / line2[1])
            x1, y1 = WIDTH2, int(-(line2[2] + line2[0] * WIDTH2) / line2[1])
            img2 = cv2.line(img2, (x0, y0), (x1, y1), color.tolist(), 1)

    NUM = int(math.floor(HEIGHT2 / SKIP))
    for idx in range(NUM):
        ptHom = np.ones(3)
        ptHom[1] = idx * SKIP
        ptHom[0] = int(WIDTH2 / 2)
        line1 = F.transpose() @ ptHom
        if line1[1] != 0:
            x0, y0 = 0, int(-line1[2] / line1[1])
            x1, y1 = WIDTH1, int(-(line1[2] + line1[0] * WIDTH1) / line1[1])
            img1 = cv2.line(img1, (x0, y0), (x1, y1), color.tolist(), 1)

    # Save intermediate images with epipolar lines
    # cv2.imwrite(os.path.join(results_dir, f"res1_pair_{i}.png"), img1)
    # cv2.imwrite(os.path.join(results_dir, f"res2_pair_{i}.png"), img2)

    # --- Rectification Steps ---
    # Compute rectifying homographies that move the epipoles to infinity
    H1 = EstimateRectifyingHomography(epipole1, F)
    H2 = EstimateRectifyingHomography(epipole2, F.transpose())

    tmp1 = cv2.warpPerspective(origImg1, H1, (WIDTH1, HEIGHT1))
    tmp2 = cv2.warpPerspective(origImg2, H2, (WIDTH2, HEIGHT2))
    # cv2.imwrite(os.path.join(results_dir, f"tmp1_pair_{i}.png"), tmp1)
    # cv2.imwrite(os.path.join(results_dir, f"tmp2_pair_{i}.png"), tmp2)

    # Modified fundamental matrix
    Fmod = np.linalg.inv(H2.transpose()) @ F @ np.linalg.inv(H1)

    # Transform point coordinates to the rectified space (homogeneous coordinates)
    NUMP = pts1.shape[0]
    pts1Hom = np.hstack([pts1, np.ones((NUMP, 1))]).T
    pts2Hom = np.hstack([pts2, np.ones((NUMP, 1))]).T
    pts1Hom = H1 @ pts1Hom
    pts2Hom = H2 @ pts2Hom

    # Adjust vertical offset and scale so that the epipolar lines are horizontal
    H1second, H2second = EstimateVerticalOffsetAndScale(Fmod, pts1Hom, pts2Hom)
    H1 = H1second @ H1
    H2 = H2second @ H2

    # Center and resize images to the expected dimensions
    EXPECTED_WIDTH = 960
    EXPECTED_HEIGHT = 600
    Corr = FinalScaleOffset(H1, WIDTH1, HEIGHT1, H2, WIDTH2, HEIGHT2, EXPECTED_WIDTH, EXPECTED_HEIGHT)
    H1 = Corr @ H1
    H2 = Corr @ H2

    # --- Compute disparity offset (doffs) for the whole images ---
    # p1 and p2 are the principal points (center) in the original images (homogeneous coordinates)
    p1 = np.array([camera_matrix_01_r[0, 2], camera_matrix_01_r[1, 2], 1])
    p2 = np.array([camera_matrix_02_r[0, 2], camera_matrix_02_r[1, 2], 1])
    p1_rect = H1 @ p1
    p2_rect = H2 @ p2
    p1_rect = p1_rect / p1_rect[2]
    p2_rect = p2_rect / p2_rect[2]
    doffs = p2_rect[0] - p1_rect[0]
    doffs_array.append(doffs)
    print(f"Pair {i} - Disparity offset (doffs): {doffs}")

    # --- Warp the images using the computed homographies ---
    transformed1 = cv2.warpPerspective(img1, H1, (EXPECTED_WIDTH, EXPECTED_HEIGHT))
    transformed2 = cv2.warpPerspective(img2, H2, (EXPECTED_WIDTH, EXPECTED_HEIGHT))
    # cv2.imwrite(os.path.join(results_dir, f"transformed1_pair_{i}.png"), transformed1)
    # cv2.imwrite(os.path.join(results_dir, f"transformed2_pair_{i}.png"), transformed2)

    result1 = cv2.warpPerspective(origImg1, H1, (EXPECTED_WIDTH, EXPECTED_HEIGHT))
    result2 = cv2.warpPerspective(origImg2, H2, (EXPECTED_WIDTH, EXPECTED_HEIGHT))
    cv2.imwrite(os.path.join(results_dir_l, f"final1_pair_{i}.png"), result1)
    cv2.imwrite(os.path.join(results_dir_r, f"final2_pair_{i}.png"), result2)

    # --- Save concatenated rectified images for comparison ---
    final_rectified = cv2.hconcat([result1, result2])
    # cv2.imwrite(os.path.join(results_dir, f"finalRectified_pair_{i}.png"), final_rectified)
    orig_concat = cv2.hconcat([origImg1, origImg2])
    # cv2.imwrite(os.path.join(results_dir, f"origRectified_pair_{i}.png"), orig_concat)

    print(f"Finished processing pair {i}\n")

Processing pair 0
Pair 0 - Epipole1: [2451.01660143  460.95621421], Epipole2: [2422.20248469  421.25459023]
pts1Hom: [[ 0.00000000e+00  1.92000000e+03  0.00000000e+00  1.92000000e+03]
 [-4.60956214e+02 -4.60956214e+02  7.39043786e+02  7.39043786e+02]
 [ 1.00000000e+00  2.16651573e-01  1.00000000e+00  2.16651573e-01]]
pts2Hom: [[ 0.00000000e+00  1.92000000e+03  0.00000000e+00  1.92000000e+03]
 [-4.60956214e+02 -4.60956214e+02  7.39043786e+02  7.39043786e+02]
 [ 1.00000000e+00  2.16651573e-01  1.00000000e+00  2.16651573e-01]]
Pts: [[    0.          -460.95621421]
 [ 8862.15583855 -2127.63844014]
 [    0.           739.04378579]
 [ 8862.15583855  3411.20895895]
 [    0.          -460.95621421]
 [ 8862.15583855 -2127.63844014]
 [    0.           739.04378579]
 [ 8862.15583855  3411.20895895]]
minPts [    0.         -2127.63844014]
maxPts [8862.15583855 3411.20895895]
Pair 0 - Disparity offset (doffs): 2.010980797052852
Finished processing pair 0

Processing pair 1
Pair 1 - Epipole1: [2869.

In [8]:
doffs_array

[np.float64(1.535659444085752),
 np.float64(-1.682309343633051),
 np.float64(1.1645537178629297),
 np.float64(0.2409340709010337),
 np.float64(-0.14552526246578168),
 np.float64(-2.2216238552981338),
 np.float64(-2.465884939152005),
 np.float64(3.8000305947960555),
 np.float64(-6.228136560543646),
 np.float64(1.8736960794601316),
 np.float64(-1.1770617676195059),
 np.float64(0.2706970963618289),
 np.float64(-1.998430780287265),
 np.float64(-0.3250591927192943),
 np.float64(2.250708690059895),
 np.float64(2.2869905305067277),
 np.float64(-2.0551508541419707),
 np.float64(0.3989786365740997),
 np.float64(0.6864191113749882),
 np.float64(-0.22705946728933668),
 np.float64(0.7237762402625378),
 np.float64(0.9295164955884161),
 np.float64(0.7785698965674896),
 np.float64(1.3843034185265424),
 np.float64(0.7945717324356565),
 np.float64(0.3987399292261671),
 np.float64(2.724863433871917),
 np.float64(8.00598087938954),
 np.float64(-0.0967906515356276),
 np.float64(-0.33293378103496707),
 np.

In [9]:
global_doff = np.median(doffs_array)
print("Global disparity offset (doff):", global_doff)

Global disparity offset (doff): 0.37672697470911487


In [16]:
import numpy as np

# Camera Matrix for Camera 1 (in pixels)
K_left = np.array([[6.3995000e+02, 0.0000000e+00, 5.0740000e+02],
       [0.0000000e+00, 6.4205000e+02, 2.8663755e+02],
       [0.0000000e+00, 0.0000000e+00, 5.0000000e-01]], dtype=np.float32)

# Translation vector for Camera 2 (in millimeters)
t_right_mm = np.array([-369.2437,0.4530,-11.3379])  # in millimeters

# Image Size (width x height)
image_width = 960  # image width in pixels
image_height = 600  # image height in pixels

# Convert the translation vector from millimeters to pixels
focal_length_x = K_left[0, 0]  # Focal length in x (pixels)
focal_length_y = K_left[1, 1]  # Focal length in y (pixels)

# Convert the translation vector to pixels
t_right_pixels = t_right_mm * np.array([focal_length_x / image_width, focal_length_y / image_height, 1])

# Output the converted translation vector in pixels
print("Translation Vector in Pixels:")
print(t_right_pixels)


Translation Vector in Pixels:
[-246.14323845    0.48474772  -11.3379    ]


In [15]:
camera_matrix_01 =	np.array([[1.2799e+03,0,1.0148e+03] , [0,1.2841e+03,573.2751] , [0,0,1]])


camera_matrix_01_r = camera_matrix_01 / 2
camera_matrix_01_r

array([[6.3995000e+02, 0.0000000e+00, 5.0740000e+02],
       [0.0000000e+00, 6.4205000e+02, 2.8663755e+02],
       [0.0000000e+00, 0.0000000e+00, 5.0000000e-01]])

## To rectify using openCV

In [2]:
import cv2
import numpy as np

# ---------------------------
# Calibration parameters
# ---------------------------

# Intrinsics for cam0 (left camera)
K_left = np.array([[1.1495e+03,0,952.4011],
                   [0,1.1448e+03,606.7985],
                   [0, 0,   1]], dtype=np.float64)

# Distortion for cam0 (left camera); MATLAB provided 2 radial coefficients and zero tangential distortion.
D_left = np.array([-0.0496,0.0361, 0, 0], dtype=np.float64)

D_right = np.array([-0.0264,0.0423, 0, 0], dtype=np.float64)
# Intrinsics for cam1 (right camera)
K_right = np.array([[1.1793e+03,0,874.7151],
                    [0,1.1727e+03,630.6559],
                    [0, 0,     1]], dtype=np.float64)

# Distortion for cam1 (right camera)
# D_71ght =np.665ay([-0.0367, 0.0265, 0, 0], dtype=np.float6# Image size in pixels (width, height)
image_size = (1920, 1200)

# Extrinsic parameters from PoseCamera2 (transformation from left to right)
R = np.array([[0.9995,0.0291,-0.0144],
              [-0.0292,0.9995,-0.0076],
              [0.0142,0.0080,0.9999]], dtype=np.float64)

T = np.array([-1.0096e+03,8.3554,78.8978], dtype=np.float64)  # in millimeters

# ---------------------------
# Read Stereo Images
# ---------------------------
img_left  = cv2.imread('../../dataset_budapest/elte_car_stereo/dev0_70_520/Dev0_Image_w1920_h1200_fn457.jpg', cv2.IMREAD_GRAYSCALE)  #queryimage # left image #, cv.IMREAD_GRAYSCALE
img_right = cv2.imread('../../dataset_budapest/elte_car_stereo/dev1_70_520/Dev1_Image_w1920_h1200_fn457.jpg', cv2.IMREAD_GRAYSCALE)


# ---------------------------
# Stereo Rectification
# ---------------------------

# The flags parameter cv2.CALIB_ZERO_DISPARITY makes the principal points of the rectified images aligned.
# alpha = -1 means that no pixels are lost (and the ROI will be computed automatically).
R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(K_left, D_left, K_right, D_right,
                                                  image_size, R, T, flags=cv2.CALIB_ZERO_DISPARITY, alpha=-1)

# Compute the undistort and rectify maps for both images.
map1_x, map1_y = cv2.initUndistortRectifyMap(K_left, D_left, R1, P1, image_size, cv2.CV_32FC1)
map2_x, map2_y = cv2.initUndistortRectifyMap(K_right, D_right, R2, P2, image_size, cv2.CV_32FC1)

# Remap the images to get rectified versions.
rectified_left = cv2.remap(img_left, map1_x, map1_y, cv2.INTER_LINEAR)
rectified_right = cv2.remap(img_right, map2_x, map2_y, cv2.INTER_LINEAR)

# ---------------------------
# Visualize the Rectified Pair
# ---------------------------
# Stack the images horizontally for visualization.
combined_rectified = np.hstack((rectified_left, rectified_right))

# Optionally, save the rectified images.
cv2.imwrite("rectified_left.png", rectified_left)
cv2.imwrite("rectified_right.png", rectified_right)

cv2.imwrite("combined.png", combined_rectified)

print("Projection Matrix P1:")
print(P1)
print("\nProjection Matrix P2:")
print(P2)

# Compute doffs as the difference between the x-coordinate of the principal points in P2 and P1
doffs = P2[0, 2] - P1[0, 2]
print("Disparity offset (doffs):", doffs)


Projection Matrix P1:
[[1.15875000e+03 0.00000000e+00 1.08403688e+03 0.00000000e+00]
 [0.00000000e+00 1.15875000e+03 6.17181320e+02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00]]

Projection Matrix P2:
[[ 1.15875000e+03  0.00000000e+00  1.08403688e+03 -1.17348074e+06]
 [ 0.00000000e+00  1.15875000e+03  6.17181320e+02  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]]
Disparity offset (doffs): 0.0


# using same rect params for all imgs

In [1]:
import os, glob
import numpy as np
import cv2

# ---------------------------
# 1) Camera parameters (your values)
# ---------------------------
K_left = np.array([[1.1495e+03, 0,          952.4011],
                   [0,          1.1448e+03, 606.7985],
                   [0,          0,          1]], dtype=np.float64)

D_left = np.array([-0.0496, 0.0361, 0, 0], dtype=np.float64)

K_right = np.array([[1.1793e+03, 0,          874.7151],
                    [0,          1.1727e+03, 630.6559],
                    [0,          0,          1]], dtype=np.float64)

D_right = np.array([-0.0264, 0.0423, 0, 0], dtype=np.float64)

R = np.array([[ 0.9995,  0.0291, -0.0144],
              [-0.0292,  0.9995, -0.0076],
              [ 0.0142,  0.0080,  0.9999]], dtype=np.float64)

T = np.array([-1.0096e+03, 8.3554, 78.8978], dtype=np.float64)

image_size = (1920, 1200)  # (w, h)

# ---------------------------
# 2) Compute rectification once
# ---------------------------
R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(
    K_left, D_left, K_right, D_right, image_size, R, T,
    flags=cv2.CALIB_ZERO_DISPARITY, alpha=-1
)

map1_x, map1_y = cv2.initUndistortRectifyMap(K_left,  D_left,  R1, P1, image_size, cv2.CV_32FC1)
map2_x, map2_y = cv2.initUndistortRectifyMap(K_right, D_right, R2, P2, image_size, cv2.CV_32FC1)

print("P1:\n", P1, "\n\nP2:\n", P2)

# ---------------------------
# 3) Batch rectify helpers
# ---------------------------
def ensure_dir(p):
    os.makedirs(p, exist_ok=True)

def list_images(folder):
    exts = ("*.png", "*.jpg", "*.jpeg", "*.bmp", "*.tif", "*.tiff")
    files = []
    for e in exts:
        files.extend(glob.glob(os.path.join(folder, e)))
    files.sort()
    return files

def rectify_folder(input_dir, output_dir, mapx, mapy, resize_to=(960, 600)):
    ensure_dir(output_dir)
    files = list_images(input_dir)
    print(f"[{input_dir}] -> [{output_dir}]  |  {len(files)} images")

    for idx, f in enumerate(files, 1):
        img = cv2.imread(f, cv2.IMREAD_UNCHANGED)
        if img is None:
            print(f"  ! Skipping unreadable file: {f}")
            continue

        rect = cv2.remap(img, mapx, mapy, interpolation=cv2.INTER_LINEAR)

        # resize to 960x600
        rect_resized = cv2.resize(rect, resize_to, interpolation=cv2.INTER_AREA)

        out_path = os.path.join(output_dir, os.path.basename(f))
        ok = cv2.imwrite(out_path, rect_resized)
        if not ok:
            print(f"  ! Failed to write: {out_path}")
        if idx % 50 == 0:
            print(f"  ...{idx} done")

# ---------------------------
# 4) Configure your folders and run
# ---------------------------
LEFT_INPUT_DIR  = "../../dataset_budapest/elte_car_stereo/dev0_70_520"
RIGHT_INPUT_DIR = "../../dataset_budapest/elte_car_stereo/dev1_70_520"

LEFT_OUTPUT_DIR  = "./rectified/dev0"
RIGHT_OUTPUT_DIR = "./rectified/dev1"

rectify_folder(LEFT_INPUT_DIR,  LEFT_OUTPUT_DIR,  map1_x, map1_y, resize_to=(960, 600))
rectify_folder(RIGHT_INPUT_DIR, RIGHT_OUTPUT_DIR, map2_x, map2_y, resize_to=(960, 600))

print("Done.")


P1:
 [[1.15875000e+03 0.00000000e+00 1.08403688e+03 0.00000000e+00]
 [0.00000000e+00 1.15875000e+03 6.17181320e+02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00]] 

P2:
 [[ 1.15875000e+03  0.00000000e+00  1.08403688e+03 -1.17348074e+06]
 [ 0.00000000e+00  1.15875000e+03  6.17181320e+02  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]]
[../../dataset_budapest/elte_car_stereo/dev0_70_520] -> [./rectified/dev0]  |  451 images
  ...50 done
  ...100 done
  ...150 done
  ...200 done
  ...250 done
  ...300 done
  ...350 done
  ...400 done
  ...450 done
[../../dataset_budapest/elte_car_stereo/dev1_70_520] -> [./rectified/dev1]  |  451 images
  ...50 done
  ...100 done
  ...150 done
  ...200 done
  ...250 done
  ...300 done
  ...350 done
  ...400 done
  ...450 done
Done.
